In [5]:
import pdb

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # 假设使用线性回归作为预测模型
%load_ext autoreload
%autoreload 2
from script.temp.tools.data_utils import data_utils
data = data_utils('config.xlsx','result.xlsx')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
data.preprocess()
print(len(data.df_result))
data.df_result = data.drop(['IOPS_Read', 'IOPS_Write', 'Bandwidth_Read', 'Bandwidth_Write'],0)
print(len(data.df_result))
data.targetLst = ['IOPS_Read']
X,y = data.dorp_and_encode()
X = X.drop(columns=['IOPS_Write', 'Bandwidth_Read', 'Bandwidth_Write'])

1336
1326


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# 准备数据
# 假设X和y已经是DataFrame类型
# 这里使用pandas的read_csv函数来示范，您可以替换成自己的数据加载方式
# data = pd.read_csv('your_data.csv')
# X = data.drop(columns=['target_column'])
# y = data['target_column']

# 示例数据
X,y = data.dorp_and_encode()
scaler = MinMaxScaler()
X = X.values.astype(float)
y = y.values.astype(float)
X_scaled = scaler.fit_transform(X)
y_scaled = y.reshape(-1, 1)/10000
y_scaled = y_scaled.squeeze()
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

#X_train_tensor ,X_train_tensor ,X_test_tensor ,y_test_tensor
# 转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [10]:
from script.temp.tools.data_utils import MLP, MPERegressionLoss

# 参数设置
input_size = X.shape[1]
output_size = 1

# 初始化模型
model = MLP([input_size, 60,30,20,15,10, output_size])

# 定义损失函数
criterion = MPERegressionLoss()
#criterion = nn.MSELoss()
# 定义优化器

# 定义数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [64]:

# 训练模型
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.001)  # weight_decay 参数即为正则化系数
num_epochs = 1000
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 评估模型
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred.squeeze(), y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Epoch [100/1000], Loss: 0.9939
Epoch [200/1000], Loss: 0.7331
Epoch [300/1000], Loss: 0.6864
Epoch [400/1000], Loss: 1.4742
Epoch [500/1000], Loss: 2.1604
Epoch [600/1000], Loss: 0.8910
Epoch [700/1000], Loss: 1.0830
Epoch [800/1000], Loss: 0.9842
Epoch [900/1000], Loss: 0.7941
Epoch [1000/1000], Loss: 0.7241
Test Loss: 1.4874


In [61]:
y_test_flat = y_test_tensor.flatten()
y_pred_flat = y_pred.flatten()
# 计算百分比误差
percentage_errors = ((y_pred_flat - y_test_flat) / y_test_flat) * 100

# 创建 DataFrame
df = pd.DataFrame({'y_test_tensor': y_test_flat, 'y_pred': y_pred_flat, 'percentage_error': percentage_errors})
mean_percentage_error = percentage_errors.abs().mean()
print("测试集:",mean_percentage_error)
y_test_flat = y_train_tensor.flatten()
y_pred_flat = model(X_train_tensor).flatten()
percentage_errors = ((y_pred_flat - y_test_flat) / y_test_flat) * 100
mean_percentage_error = percentage_errors.abs().mean()
print("训练集:",mean_percentage_error)

测试集: tensor(150.9725)
训练集: tensor(152.7999, grad_fn=<MeanBackward0>)


In [36]:
# 计算均方误差（MSE）
mse_loss = nn.MSELoss()
mse = mse_loss(y_pred.squeeze(), y_test_tensor)

# 计算平均百分比误差（MPE）
mpe = torch.mean(torch.abs((y_test_tensor - y_pred) / y_test_tensor)) * 100

# 打印结果
print(f'MSE: {mse.item():.4f}')
print(f'MPE: {mpe.item():.4f}')

MSE: 5.4452
MPE: 3212.8315


In [81]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

# 实例化决策树回归模型
tree_reg = DecisionTreeRegressor()

# 在训练集上训练模型
tree_reg.fit(X_train_tensor, y_train_tensor)

# 在训练集和测试集上进行预测
y_train_pred = tree_reg.predict(X_train_tensor)
y_test_pred = tree_reg.predict(X_test_tensor)

# 计算训练集和测试集上的平均绝对误差
train_mae = mean_absolute_error(y_train_tensor, y_train_pred)
test_mae = mean_absolute_error(y_test_tensor, y_test_pred)

# 打印训练集和测试集上的平均绝对误差
print("训练集上的平均绝对误差:", train_mae)
print("测试集上的平均绝对误差:", test_mae)


训练集上的平均绝对误差: 0.0
测试集上的平均绝对误差: 0.39446816818645974


In [82]:
print("树的最大深度:", tree_reg.max_depth)
print("内部节点再划分所需的最小样本数:", tree_reg.min_samples_split)
print("叶节点所需的最小样本数:", tree_reg.min_samples_leaf)
print("寻找最佳划分时考虑的特征数量:", tree_reg.max_features)
print("划分时使用的评价标准:", tree_reg.criterion)


树的最大深度: None
内部节点再划分所需的最小样本数: 2
叶节点所需的最小样本数: 1
寻找最佳划分时考虑的特征数量: None
划分时使用的评价标准: squared_error


In [89]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.tree import plot_tree

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 实例化决策树回归模型
tree_reg = DecisionTreeRegressor(random_state=42)

# 在训练集上训练模型
tree_reg.fit(X_train, y_train)

# 在训练集和测试集上进行预测
y_train_pred = tree_reg.predict(X_train)
y_test_pred = tree_reg.predict(X_test)

# 计算均方误差和平均百分比误差
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_mape = np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100
test_mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100

# 打印训练集和测试集的均方误差和平均百分比误差
print("训练集均方误差:", train_mse)
print("测试集均方误差:", test_mse)
print("训练集平均百分比误差:", train_mape)
print("测试集平均百分比误差:", test_mape)

# 展示决策树模型
#plt.figure(figsize=(20, 10))
#plot_tree(tree_reg, filled=True, feature_names=X.columns)
#plt.show()


训练集均方误差: 0.0
测试集均方误差: 599531901.634528
训练集平均百分比误差: 3865.0711596859096
测试集平均百分比误差: 3061.2581464458726


In [94]:
X_train_tensor ,X_train_tensor ,X_test_tensor ,y_test_tensor

IndexError: index 50 is out of bounds for axis 0 with size 50

In [99]:
print(X_train_tensor.shape,X_train_tensor.shape ,X_test_tensor.shape ,y_test_tensor.shape)

torch.Size([1060, 103]) torch.Size([1060, 103]) torch.Size([266, 103]) torch.Size([266])


In [1]:
import numpy as np
import pdb
from sklearn.tree import DecisionTreeRegressor

# 遗传算法参数
population_size = 100
num_generations = 100
mutation_rate = 0.3
tournament_size = 5

# 定义适应度函数，这里我们使用决策树的准确率作为适应度
def fitness_function(selected_features, X_train, y_train, X_test, y_test):
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    model = DecisionTreeRegressor()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    #accuracy = model.score(X_test_selected, y_test)
    accuracy = np.mean(np.abs(y_test - y_pred) / y_test) * 100
    return accuracy

# 初始化种群
def initialize_population(population_size, num_features):
    return np.random.choice([0, 1], size=(population_size, num_features)).astype(bool)
    #return np.ones((population_size,num_features)).astype(bool)

# 选择操作（锦标赛选择）
def tournament_selection(population, fitness_values, tournament_size):
    tournament_indices = np.random.choice(len(population), size=tournament_size, replace=False)
    tournament_fitness = fitness_values[tournament_indices]
    winner_index = tournament_indices[np.argmax(tournament_fitness)]
    return population[winner_index]

# 交叉操作（单点交叉）
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    #pdb.set_trace()
    #print("c1,",child1)
    #print("p1:",parent1)
    return child1, child2

# 变异操作
def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if np.random.rand() < mutation_rate:
            individual[i] = 1 - individual[i]  # 变异操作是翻转基因位
            #print("invdividual",individual)
    return individual

# 遗传算法主循环
def genetic_algorithm(X_train, y_train, X_test, y_test):
    num_features = X_train.shape[1]
    population = initialize_population(population_size, num_features)
    best_fitness = -np.inf
    best_individual = None

    for generation in range(num_generations):
        fitness_values = np.array([fitness_function(individual, X_train, y_train, X_test, y_test) for individual in population])

        # 更新最佳个体
        generation_best_index = np.argmax(fitness_values)
        if fitness_values[generation_best_index] > best_fitness:
            best_fitness = fitness_values[generation_best_index]
            best_individual = population[generation_best_index]

        # 选择新种群
        new_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population, fitness_values, tournament_size)
            parent2 = tournament_selection(population, fitness_values, tournament_size)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = np.array(new_population)

    return best_individual

# 将 PyTorch 张量转换为 NumPy 数组
X_train = X_train_tensor.numpy()
y_train = y_train_tensor.numpy()
X_test = X_test_tensor.numpy()
y_test = y_test_tensor.numpy()

# 运行遗传算法选择特征
selected_features = genetic_algorithm(X_train, y_train, X_test, y_test)



In [228]:
# 计算误差

# 根据选择的特征重新训练决策树模型
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

model = DecisionTreeRegressor()
model.fit(X_train_selected, y_train)

# 使用测试集进行预测
y_pred = model.predict(X_test_selected)

# 在此之后，你可以使用各种评估指标来评估模型的性能
train_error = np.mean(np.abs(y_train - model.predict(X_train_selected)) / y_train) * 100
test_error = np.mean(np.abs(y_test - y_pred) / y_test) * 100

# 打印误差和选择的特征
print("训练集百分比误差: {:.2f}%".format(train_error))
print("测试集百分比误差: {:.2f}%".format(test_error))
print("选择的特征索引:", selected_features)


训练集百分比误差: 0.00%
测试集百分比误差: 15500.72%
选择的特征索引: [False False  True  True False False False  True  True False False  True
 False  True  True  True False  True False  True False  True  True False
 False False  True False  True False  True  True False  True  True False
 False  True  True  True  True False False False False  True  True False
 False False  True False False False False False  True  True  True  True
  True False False  True False False  True  True  True  True False  True
  True  True  True False  True False False False False False  True  True
 False  True  True False False  True False False  True  True False False
 False  True  True  True  True False False]


In [207]:
X_train_selected

array([[0.5889676 , 0.42036074, 0.71428573, ..., 0.        , 1.        ,
        0.        ],
       [0.19993305, 0.24112625, 0.5714286 , ..., 0.        , 1.        ,
        0.        ],
       [0.3902107 , 0.8689414 , 0.42857143, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.10595768, 0.8421125 , 0.14285715, ..., 1.        , 0.        ,
        0.        ],
       [0.5512777 , 0.42654246, 0.2857143 , ..., 0.        , 0.        ,
        1.        ],
       [0.6556804 , 0.8759798 , 0.2857143 , ..., 1.        , 0.        ,
        1.        ]], dtype=float32)

In [109]:
X_ori,y_ori = data.dorp_and_encode()
print(X_ori.columns)
print(len(selected_features))

Index(['PCIe_Lane_Bandwidth', 'PCIe_Lane_Count', 'SATA_Processing_Delay',
       'IO_Queue_Depth', 'Queue_Fetch_Size', 'Data_Cache_Capacity',
       'Data_Cache_DRAM_Row_Size', 'Data_Cache_DRAM_Data_Rate',
       'Data_Cache_DRAM_Data_Busrt_Size', 'Data_Cache_DRAM_tRCD',
       ...
       'Flash_Technology_SLC', 'Flash_Technology_TLC',
       'CMD_Suspension_Support_ERASE', 'CMD_Suspension_Support_NONE',
       'CMD_Suspension_Support_PROGRAM',
       'CMD_Suspension_Support_PROGRAM_ERASE',
       'Address_Distribution_RANDOM_UNIFORM', 'Address_Distribution_STREAMING',
       'Request_Size_Distribution_FIXED', 'Request_Size_Distribution_NORMAL'],
      dtype='object', length=103)
103


In [116]:
print(X_test_selected.shape)

(266, 103)


In [217]:
from sklearn.ensemble import RandomForestRegressor
selected_features_m = np.ones(103).astype(bool)

flst =[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,21,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,62,63,64,65,66]
for i in flst:
    selected_features_m[i] = False
print(selected_features_m)
selected_features_m
#i = 8
#selected_features_m[i] = 0
X_train_selected = X_train[:, selected_features_m]
X_test_selected = X_test[:, selected_features_m]


#X_train_selected = X_train
#X_test_selected = X_test
#X_test_selected = X_test
#model = RandomForestRegressor()
model = DecisionTreeRegressor()
model.fit(X_train_selected, y_train)

# 使用测试集进行预测
y_pred = model.predict(X_test_selected)
# 计算误差
train_error = np.mean(np.abs(y_train - model.predict(X_train_selected)) / y_train) * 100
test_error = np.mean(np.abs(y_test - y_pred) / y_test) * 100

# 打印误差和选择的特征
print("训练集百分比误差: {:.2f}%".format(train_error))
print("测试集百分比误差: {:.2f}%".format(test_error))
print("选择的特征索引:", selected_features)


[ True False False False False False False False False False False False
 False False False False  True False  True  True False False False False
 False False False False False False False False False False False False
 False False False False False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False False False False False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]
训练集百分比误差: 0.00%
测试集百分比误差: 26.72%
选择的特征索引: [ True False  True False  True False False False False False False  True
  True  True False  True False  True False  True  True False  True False
  True False  True False False  True  True  True False False False  True
 False False  True False  True  True  True  True  True  True  True False
  True  True  True False False False F

In [126]:
def initialize_population(population_size, num_features):
    return np.random.choice([0, 1], size=(population_size, num_features))
z = initialize_population(3,5)
k = np.ones((3,5))
print(z,"\n",k)

[[0 1 1 1 1]
 [1 1 0 0 0]
 [0 0 0 0 1]] 
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [176]:
select = np.zeros(103).astype(bool)
select[0] = 1
X_train[:,select]
X_train_selected
#X_train_selected.shape

array([[0.15789473, 0.23004067, 0.41458333, ..., 0.        , 0.        ,
        1.        ],
       [0.36842105, 0.86501396, 0.95208335, ..., 0.        , 0.        ,
        1.        ],
       [0.7368421 , 0.7748926 , 0.32291666, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.05263158, 0.5017592 , 0.94375   , ..., 0.        , 0.        ,
        0.        ],
       [0.7368421 , 0.13394076, 0.3625    , ..., 1.        , 0.        ,
        0.        ],
       [0.42105263, 0.20556554, 0.45      , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [169]:
X_train
#X_train.shape

array([[0.5889676 , 0.15789473, 0.42036074, ..., 1.        , 1.        ,
        0.        ],
       [0.19993305, 0.36842105, 0.24112625, ..., 0.        , 1.        ,
        0.        ],
       [0.3902107 , 0.7368421 , 0.8689414 , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.10595768, 0.05263158, 0.8421125 , ..., 1.        , 1.        ,
        0.        ],
       [0.5512777 , 0.7368421 , 0.42654246, ..., 0.        , 0.        ,
        1.        ],
       [0.6556804 , 0.42105263, 0.8759798 , ..., 0.        , 0.        ,
        1.        ]], dtype=float32)